### This is the basic idea and code snippets of our MWPM decoder.

#### 1. Read the testpoint

In [1]:
import numpy as np

# error_x, error_z, syndrome_x, syndrome_z,sizeX,sizeY = read_test_file('test_file.npz')
def read_test_file(filename):
    file_path = "../code/data/input/" + filename
    data = np.load(file_path, allow_pickle=True)
    file_content = data['arr_0'].item()
    return file_content["sizeX"], file_content["sizeY"] ,file_content["syndrome_x"], file_content["syndrome_z"]

def write_result_file(consume_time,correct_x,correct_z,filename):
    file_content = {
        "time": consume_time,
        "correct_x":correct_x,
        "correct_z":correct_z,
    }
    np.savez("../code/data/output/"+filename, file_content)
    return file_content

X,Y,synX,synZ = read_test_file('test1.npz')

5 5


#### 2. Build Matching graph

because we will use dijkstra algorithm to connect different defect point, so first in the matching graph we need to make syndrome inputable for dijkstra.

#### 3. Build Syndrome graph

using dijkstra / localdijkstra to build a syndrome graph

#### 4. Blossom
use blossom algorithm to find perfect matching

#### 5. output correct graph
Need to output every edge(sizeX\*sizeY\*2) on the matching graph, tell whether we detect error there.